In [63]:
import requests
import json
import pandas as pd
from datetime import datetime
today = datetime.now()

def get_sport_odds(api_key, sport_key='americanfootball_ncaaf', regions='us'):
    """
    Fetches sports odds data from the Odds API.

    Args:
        api_key (str): API key for authentication.
        sport_key (str): Key identifying the specific sport.
        regions (str): Region for which odds are to be fetched.

    Returns:
        dict: JSON data containing odds information for the specified sport.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data):
    """
    Processes the odds data and creates a DataFrame.

    Args:
        data (dict): JSON data containing odds information.

    Returns:
        pd.DataFrame: Processed data in the form of a DataFrame.
    """
    data_list = []
    for game in data:
        game_date = game['commence_time']
        sport = game['sport_title']
        home_team = game['home_team']
        away_team = game['away_team']
        for bookmaker in game['bookmakers']:
            bookmaker_name = bookmaker['title']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        team_name = outcome['name']
                        if team_name.lower() == home_team.lower():
                            team_type = 'home'
                        elif team_name.lower() == away_team.lower():
                            team_type = 'away'
                        else:
                            team_type = 'unknown'
                        row = {
                            'date': game_date,
                            'sport': sport,
                            'home_team': home_team,
                            'away_team': away_team,
                            'bookmaker': bookmaker_name,
                            'team_type': team_type,
                            'price': outcome['price']
                        }
                        data_list.append(row)
    return pd.DataFrame(data_list)

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.

    Args:
        df (pd.DataFrame): Raw DataFrame containing odds information.

    Returns:
        pd.DataFrame: Pivoted and cleaned DataFrame.
    """
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)
    return pivoted_df

# Example usage
API_KEY = '287f26ecf5b58ab73781cfaf78310c08'
sport_key = 'americanfootball_ncaaf'
data = get_sport_odds(API_KEY, sport_key)
if data:
    processed_data = process_odds_data(data)
    pivoted_data = pivot_and_clean_data(processed_data)

pivoted_data.to_csv(f"../../data/upcoming-{today}.csv")